In [ ]:
# Load and format the data provided by Kaggle.

import pandas as pd
import dataset as ds
import preprocessing as pp

X_dataset, y_dataset = ds.load_training_set()
X_testset = ds.load_test_set()

df = pd.merge(X_dataset.reset_index(), X_testset.reset_index(), how='outer').set_index(ds.ID_COLUMN_NAME)

df = pp.manual_fixes(df)
df = pp.format_name(df)
df = pp.add_ticket_number_column(df)
df = df.reset_index()
df.shape

In [ ]:
df.loc[df.Ticket == '17582']

In [ ]:
# Load and format the extra data we crawled.
import data.integration.merge as merge

edf = merge.import_extra_data()
edf = merge.apply_post_processing(edf)
edf.shape

In [ ]:
# Key: Kaggle PassengerId, value: extra data PassengerId
matches = {}

In [ ]:
# Leave behind the invalid ticket numbers from the Kaggle data.
df1 = df.loc[df['TicketNumber'] > 0]
ticket_nrs_1 = set(df1.TicketNumber.unique())
print('{}/{} rows, {} unique tickets'.format(len(df1), len(df), len(ticket_nrs_1)))
df1.shape

In [ ]:
# These are passengers with ticket number "LINE". Let's try to match them manually.
valid_ticket_ids = set(df1.PassengerId.values)
line_ticket_ids = set(df.PassengerId.values) - valid_ticket_ids
line_ticket_df = df.loc[df.PassengerId.isin(line_ticket_ids)]

# Apparently Andrew John Shannon used the name Lionel Leonard for unknown reasons.
# See: https://www.encyclopedia-titanica.org/titanic-victim/lionel-leonard.html
matches[180] = 1223
matches[272] = 2148
matches[303] = 690
matches[598] = 688

line_ticket_df.merge(edf, how='left', left_on='UnmarriedLastName', right_on='LastName')[['PassengerId_x', 'UnmarriedFirstName', 'FirstName_y', 'UnmarriedLastName', 'LastName_y', 'PassengerId_y', 'UrlId']]

In [ ]:
# Leave behind the invalid ticket numbers from the extra data.
df2 = edf.loc[edf['TicketNumber'] > 0]
ticket_nrs_2 = set(df2.TicketNumber.unique())
print('{}/{} rows, {} unique tickets'.format(len(df2), len(edf), len(ticket_nrs_2)))
df2.shape

In [ ]:
# What is the overlap between these two?
common_ticket_nrs = ticket_nrs_1 & ticket_nrs_2
print('{} common tickets on ({}, {})'.format(len(common_ticket_nrs), len(ticket_nrs_1), len(ticket_nrs_2)))

In [ ]:
# Count the ticket numbers assigned to a single passenger in both dataframes: on these we can operate a 1-to-1 merge.

from tqdm import tqdm

single_passenger_ticket_nrs = []
for ticket_nr in tqdm(common_ticket_nrs, leave=False, disable=False):
    sub_df1 = df1.loc[df1.TicketNumber == ticket_nr]
    sub_df2 = df2.loc[df2.TicketNumber == ticket_nr]
    
    len1, len2 = len(sub_df1), len(sub_df2)
    if len1 == 1 and len1 == len2:
        single_passenger_ticket_nrs.append(ticket_nr)
        id1 = sub_df1.iloc[0]['PassengerId']
        id2 = sub_df2.iloc[0]['PassengerId']
        matches[id1] = id2

multiple_passengers_ticket_nrs = common_ticket_nrs - set(single_passenger_ticket_nrs)

print('{}/{} ticket numbers assigned to a single passenger in both data frames'.format(len(single_passenger_ticket_nrs), len(common_ticket_nrs)))
print('{}/{} passengers matched'.format(len(matches), len(df)))

In [ ]:
def manually_match_first_names(df):
    df = df.set_index('PassengerId')
    
    # These matches proposals are auto-generated.
#     df.loc[600, 'UnmarriedFirstName'] = 'lucy' # cosmo
    df.loc[541, 'UnmarriedFirstName'] = 'catherine' # harriet
    df.loc[639, 'UnmarriedFirstName'] = 'maija' # maria
    df.loc[280, 'UnmarriedFirstName'] = 'rhoda' # rosa
    df.loc[499, 'UnmarriedFirstName'] = 'bess' # bessie
    df.loc[1007, 'UnmarriedFirstName'] = 'dimitrios' # demetrios
    df.loc[1019, 'UnmarriedFirstName'] = 'alice' # alicia
    df.loc[242, 'UnmarriedFirstName'] = 'catherine' # katherine
    df.loc[53, 'UnmarriedFirstName'] = 'myra' # myna
#     df.loc[333, 'UnmarriedFirstName'] = 'margaret' # george
#     df.loc[17, 'UnmarriedFirstName'] = 'francis' # eugene
    df.loc[276, 'UnmarriedFirstName'] = 'cornelia' # kornelia
    df.loc[1241, 'UnmarriedFirstName'] = 'ellen' # nellie
#     df.loc[792, 'UnmarriedFirstName'] = 'william' # alfred
    df.loc[1095, 'UnmarriedFirstName'] = 'winnifred' # winifred
#     df.loc[490, 'UnmarriedFirstName'] = 'neville' # eden
    df.loc[1230, 'UnmarriedFirstName'] = 'albert' # herbert
    df.loc[270, 'UnmarriedFirstName'] = 'nellie' # amelia
    df.loc[110, 'UnmarriedFirstName'] = 'bridget' # bertha
#     df.loc[832, 'UnmarriedFirstName'] = 'sibley' # george
#     df.loc[521, 'UnmarriedFirstName'] = 'mary' # anne
    df.loc[437, 'UnmarriedFirstName'] = 'dollina' # doolina
    
    # These matches proposals are manually written.
    df.loc[66, 'UnmarriedFirstName'] = 'jirjis' # jirjis
    df.loc[710, 'UnmarriedFirstName'] = 'halim' # william
    df.loc[1117, 'UnmarriedFirstName'] = 'aminah' # omine
    
    df.loc[449, 'UnmarriedFirstName'] = 'mariya' # marie
    df.loc[470, 'UnmarriedFirstName'] = 'hilanah' # helene
    df.loc[645, 'UnmarriedFirstName'] = 'uwjiniya' # eugenie
    df.loc[859, 'UnmarriedFirstName'] = 'latifah' # latifa
    
    df.loc[208, 'UnmarriedFirstName'] = 'nasif' # nassef
    df.loc[732, 'UnmarriedFirstName'] = 'husayn' # houssein
    
    df.loc[259, 'UnmarriedFirstName'] = 'annie' # anna

    df = df.reset_index()
    
    return df

In [ ]:
from unidecode import unidecode

num_exact_matches = 0

displayed = 0
max_display = 10

# Format the first names as much as possible, as we are going to try to use those to match passengers.
df1.UnmarriedFirstName = df1.UnmarriedFirstName.apply(unidecode).str.split(expand=True)[0].str.lower()
df2.FirstName = df2.FirstName.apply(unidecode).str.split(expand=True)[0].str.lower()

# Manually fix some first names so that the Kaggle data matches my extra data.
df1 = manually_match_first_names(df1)

for ticket_nr in tqdm(multiple_passengers_ticket_nrs, disable=False):
    sub_df1 = df1.loc[df1.TicketNumber == ticket_nr]
    sub_df2 = df2.loc[df2.TicketNumber == ticket_nr]
    
    len1, len2 = len(sub_df1), len(sub_df2)
    if len1 == len2:
        # Same amount of passengers: there is a chance for a 1-to-1 mapping.

        # Try to merge on first name.
        merge_on_name_df = sub_df1.merge(sub_df2, how='outer', left_on=['UnmarriedFirstName'], right_on=['FirstName'])
        if len(merge_on_name_df) == len1:
            # All the passengers related to this ticket successfully matched!
            num_exact_matches += 1
            ids1 = merge_on_name_df['PassengerId_x'].tolist()
            ids2 = merge_on_name_df['PassengerId_y'].tolist()
            matches.update(dict(zip(ids1, ids2)))
            continue
        
        # Merging on first names is still ambiguous: use the title.
        merge_on_title_df = sub_df1.merge(sub_df2, how='outer', on='Title')
        if len(merge_on_title_df) == len1:
            # All the passengers related to this ticket successfully matched!
            num_exact_matches += 1
            ids1 = merge_on_title_df['PassengerId_x'].tolist()
            ids2 = merge_on_title_df['PassengerId_y'].tolist()
            matches.update(dict(zip(ids1, ids2)))
            continue
        
        # First name and titles are still ambiguous. Try to use only the first letters of the first name. -> worked only in 1 case
        
        # Not all the passenger related to this ticket could be matched. Some probably remained unmatched and others were
        # matched multple times. If a 1-on-1 match occurred, the PassengerId should be repeated only once.
        unique_ids_df = merge_on_name_df.dropna(axis='index', subset=['PassengerId_x', 'PassengerId_y'], how='any')
        unique_ids_df = unique_ids_df.drop_duplicates('PassengerId_x', keep=False)
        unique_ids_df = unique_ids_df.drop_duplicates('PassengerId_y', keep=False)
        
        assert len(unique_ids_df) == len(unique_ids_df.PassengerId_x.unique())
        assert len(unique_ids_df) == len(unique_ids_df.PassengerId_y.unique())
        
        ids1 = unique_ids_df['PassengerId_x'].tolist()
        ids2 = unique_ids_df['PassengerId_y'].tolist()
        # print(unique_ids_df[['PassengerId_x', 'UnmarriedFirstName', 'FirstName_y', 'PassengerId_y']])
        matches.update(dict(zip(ids1, ids2)))
        
        # Check which passengers remain un-matched.
        unmatched_1 = merge_on_name_df.loc[merge_on_name_df.FirstName_y.isna()]
        unmatched_2 = merge_on_name_df.loc[merge_on_name_df.UnmarriedFirstName.isna()]
        
#         if len(unmatched_1) == 1 and len(unmatched_2) == 1:
#             passenger_id = unmatched_1.PassengerId.values[0]
#             kaggle_name = unmatched_1.UnmarriedFirstName.values[0]
#             extra_name = unmatched_2.FirstName_y.values[0]
#             print("df.loc[{}, 'UnmarriedFirstName'] = '{}' # {}".format(int(passenger_id), extra_name, kaggle_name))
        
#         if len(unmatched_1) > 0 or len(unmatched_2) > 0:
#             print('============================')
#             print(unmatched_2[['PassengerId', 'UnmarriedFirstName', 'FirstName_y']])
#             print(unmatched_1[['PassengerId', 'UnmarriedFirstName', 'FirstName_y']])

print('{}/{} successful ticket matches'.format(num_exact_matches, len(multiple_passengers_ticket_nrs)))
print('{} ticket numbers left'.format(len(multiple_passengers_ticket_nrs) - num_exact_matches))
print('{}/{} passengers matched ({} left)'.format(len(matches), len(df), len(df) - len(matches)))

In [ ]:
# Try to manually match the missing passengers.

matches[17] = 1125
matches[75] = 1556
matches[170] = 810
matches[194] = 1964
matches[306] = 1512
matches[312] = 2061
matches[319] = 2185
matches[490] = 1640
matches[521] = 2004
matches[551] = 2134
matches[684] = 515
matches[691] = 1673
matches[699] = 1337
matches[731] = 1509
matches[832] = 2040
matches[946] = 845
matches[1015] = 248
matches[792] = 485
matches[573] = 1711
matches[1031] = 513
matches[1044] = 1300
matches[1066] = 59
matches[1080] = 1192
matches[1198] = 30
matches[1204] = 1182
matches[1271] = 61
matches[1252] = 1187
matches[39] = 1384
matches[334] = 1386
matches[1037] = 1385
matches[19] = 1387
matches[149] = 963
matches[738] = 1875
matches[780] = 2043
matches[782] = 1675
matches[916] = 2058
matches[857] = 2186
matches[600] = 1692
matches[557] = 1690
matches[333] = 524
matches[888] = 1748

# Double check that the mathces are 1-on-1
num_matches = len(matches)
num_unique_values = len(set(matches.values()))
assert num_matches == num_unique_values

unmatched_ids1 = set(df.PassengerId.values) - set(matches.keys())
unmatched_ids2 = set(edf.PassengerId.values) - set(matches.values())
unmatched_passengers_df1 = df.loc[df.PassengerId.isin(unmatched_ids1)]
unmatched_passengers_df2 = edf.loc[edf.PassengerId.isin(unmatched_ids2)]

print('{}/{} passengers matched ({} left)'.format(len(matches), len(df), len(df) - len(matches)))

unmatched_passengers_df1.merge(unmatched_passengers_df2, how='left', left_on='UnmarriedLastName', right_on='LastName')[['PassengerId_x', 'UnmarriedFirstName', 'FirstName_y', 'UnmarriedLastName', 'LastName_y', 'PassengerId_y', 'TicketNumber_x', 'TicketNumber_y', 'Age_x', 'Age_y', 'UrlId']]

In [ ]:
import os
import json

jsonizable_matches = {
    int(k): int(v)
    for k, v in matches.items()
}

matches_filepath = os.path.join(os.environ['HOME'], 'kaggle/titanic/data/integration', 'matches.json')
with open(matches_filepath, 'w') as f:
    json.dump(jsonizable_matches, f, indent=2)
